In [1]:

import os
# demonstration of calculating metrics for a neural network model using sklearn
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

# example of random oversampling to balance the class distribution
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from keras.layers import Bidirectional
from keras.utils import to_categorical
from matplotlib import pyplot
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint

import tensorflow as tf

from scipy import stats

import datetime;
import warnings
warnings.filterwarnings("ignore")

2024-02-23 22:20:28.833360: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-23 22:20:29.220283: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-23 22:20:30.281646: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-02-23 22:20:30.281779: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
# configs
#EPOCHS = 5
BATCH_SIZE = 32

VERBOSE = 0
baseFolder = "../data_2019_processed/"

fileSufixTrain = "_transformed_smote" # _transformed_smote unbalanced file sufix is empty
fileSufixTrain = "_transformed" # unbalanced file sufix is empty
outputFileSufixTrain = "unb" # unbalanced file sufix is unb

# selected features
inputFeatures = ["activity","location","day_of_week",
                 "light","phone_lock","proximity",
                 "sound","time_to_next_alarm", "minutes_day"]
outputClasses = ["awake","asleep"]
#outputClasses = ["class"]

TIME_SERIES_SIZE = 4 
TIME_STEP_SHIFT = 4

NN_type = 'LSTM_BI
UNITS_NUMBER = "128";
EPOCHS_ARRAY_TEST = [5,15,30,50,80,100,120,150,200,250,300]

generalName = "result_trad_"+str(NN_type)+"_"+outputFileSufixTrain+"_batch_size_"+str(BATCH_SIZE)+"_window_"+str(TIME_SERIES_SIZE)+"-"+str(TIME_STEP_SHIFT)

outputMetricFile = generalName+".csv"
outputMetricFilePartials = generalName+"_partial.csv"
outputCheckpointFolder = generalName+"_checkpoints"
checkpointName_prefix = "checkpoint_epoch_{epoch}.hdf5"

In [3]:
print("Checking whether the checkpoint folder exists or not")
isExist = os.path.exists(outputCheckpointFolder)
if not isExist:
    # Create a new directory because it does not exist
    os.makedirs(outputCheckpointFolder)
    print("The new checkpoint directory is created!")
else:
    print("The checkpoint directory exists!")

Checking whether the checkpoint folder exists or not
The new checkpoint directory is created!


In [4]:
outputMetricFile

'result_trad_LSTM_unb_batch_size_32_window_60-30.csv'

In [5]:
# y_test     = Array with real values
# yhat_probs = Array with predicted values
def printMetrics(y_test,yhat_probs):
    # predict crisp classes for test set deprecated
    #yhat_classes = model.predict_classes(X_test, verbose=0)
    #yhat_classes = np.argmax(yhat_probs,axis=1)
    yhat_classes = yhat_probs.round()
    # accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(y_test, yhat_classes)
    print('Accuracy: %f' % accuracy)
    # precision tp / (tp + fp)
    precision = precision_score(y_test, yhat_classes)
    print('Precision: %f' % precision)
    # recall: tp / (tp + fn)
    recall = recall_score(y_test, yhat_classes)
    print('Recall: %f' % recall)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(y_test, yhat_classes)
    print('F1 score: %f' % f1)
    # kappa
    kappa = cohen_kappa_score(y_test, yhat_classes)
    print('Cohens kappa: %f' % kappa)
    # ROC AUC
    auc = roc_auc_score(y_test, yhat_probs)
    print('ROC AUC: %f' % auc)
    # confusion matrix
    print("\Confusion Matrix")
    matrix = confusion_matrix(y_test, yhat_classes)
    print(matrix)
    
    array = []
    results = dict()
    results['accuracy'] = accuracy
    results['precision'] = precision
    results['recall'] = recall
    results['f1_score'] = f1
    results['cohen_kappa_score'] = kappa
    results['roc_auc_score'] = auc
    #results['matrix'] = np.array(matrix,dtype=object)
    results['matrix'] = 0
    results['TP'] = matrix[0][0]
    results['FP'] = matrix[0][1]
    results['FN'] = matrix[1][0]
    results['TN'] = matrix[1][1]
    
    array.append(accuracy)
    array.append(precision)
    array.append(recall)
    array.append(f1)
    array.append(kappa)
    array.append(auc)
    #array.append(np.array(matrix,dtype=object)))
    array.append(0)
    array.append(matrix[0][0]) # TP
    array.append(matrix[0][1]) # FP
    array.append(matrix[1][0]) # FN
    array.append(matrix[1][1]) # TN
    
    return results, array

def showGlobalMetrics(metrics):
    accuracy,precision,recall,f1_score,cohen_kappa_score,roc_auc_score = 0,0,0,0,0,0
    for metric in metrics:
        accuracy = accuracy + metric['accuracy']
        precision = precision + metric['precision']
        recall = recall + metric['recall']
        f1_score = f1_score + metric['f1_score']
        cohen_kappa_score = cohen_kappa_score + metric['cohen_kappa_score']
        roc_auc_score = roc_auc_score + metric['roc_auc_score']
        
    # mean
    size = len(metrics)
    print(size)
    accuracy = accuracy / size
    precision = precision / size
    recall = recall / size
    f1_score = f1_score / size
    cohen_kappa_score = cohen_kappa_score / size
    roc_auc_score = roc_auc_score / size
    
    #show:\
    print("accuracy: ",accuracy)
    print("precision: ",precision)
    print("recall: ",recall)
    print("f1_score: ",f1_score)
    print("cohen_kappa_score: ",cohen_kappa_score)
    print("roc_auc_score: ",roc_auc_score)
    
    return [accuracy,precision,recall,f1_score,cohen_kappa_score,roc_auc_score]
    
def transform_data_type(dataframe):
    
    # transform inputs
    for column in inputFeatures:
        dataframe[column] = dataframe[column].astype('float32')
    
    # transform outputs
    for column in outputClasses:
        dataframe[column] = dataframe[column].astype('float32')
    
    return dataframe

# one-hot encoding function
def transform_output_nominal_class_into_one_hot_encoding(dataset):
    # create two classes based on the single class
    one_hot_encoded_data = pd.get_dummies(dataset['class'])
    #print(one_hot_encoded_data)
    dataset['awake'] = one_hot_encoded_data['awake']
    dataset['asleep'] = one_hot_encoded_data['asleep']
    
    return dataset

# one-hot encoding function
def transform_output_numerical_class_into_one_hot_encoding(dataset):
    # create two classes based on the single class
    one_hot_encoded_data = pd.get_dummies(dataset['class'])
    #print(one_hot_encoded_data)
    dataset['awake'] = one_hot_encoded_data[0]
    dataset['asleep'] = one_hot_encoded_data[1]
    
    return dataset


def create_dataset_time_series_with_one_output(X, y, window_time_steps=1, shift_step=1):
    Xs, ys = [], []
    for i in range(0, len(X) - window_time_steps, shift_step):
        v = X.iloc[i:(i + window_time_steps)].values
        labels = y.iloc[i: i + window_time_steps]
        Xs.append(v)        
        ys.append(stats.mode(labels)[0][0])
        
    if len(y.columns) == 1:
        return np.array(Xs), np.array(ys).reshape(-1, 1)
    else:
        return np.array(Xs), np.array(ys).reshape(-1, len(y.columns))
    
def create_dataset_time_series_with_one_output_foward(X, y, window_time_steps=1, shift_step=1):
    Xs, ys = [], []
    for i in range(0, len(X) - window_time_steps, shift_step):
        valuesX = X.iloc[i:(i + window_time_steps)].values # values
        valuesY = y.iloc[i: i + window_time_steps].values # labels
        
        Xs.append(valuesX)
        ys.append(valuesY[window_time_steps-1]) # append only the last value
        
    if len(y.columns) == 1:
        return np.array(Xs), np.array(ys).reshape(-1, 1)
    else:
        return np.array(Xs), np.array(ys).reshape(-1, len(y.columns))

In [6]:
# client datasets used on the training process (75% of data)
trainFolders =  ['0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs',
                '0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA', 
                '2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0', 
                '2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6TrLe89ys', 
                #['5FLZBTVAPwdq9QezHE2sVCJIs7p+r6mCemA2gp9jATk'], #does not have the file
                '7EYF5I04EVqisUJCVNHlqn77UAuOmwL2Dahxd3cA', 
                'a9Qgj8ENWrHvl9QqlXcIPKmyGMKgbfHk9Dbqon1HQP4', 
                'ae4JJBZDycEcY8McJF+3BxyvZ1619y03BNdCxzpZTc', 
                'Ch3u5Oaz96VSrQbf0z31X6jEIbeIekkC0mwPzCdeJ1U', 
                'CH8f0yZkZL13zWuE9ks1CkVJRVrr+jsGdUXHrZ6YeA', 
                'DHO1K4jgiwZJOfQTrxvKE2vn7hkjamigroGD5IaeRc', 
                #'DHPqzSqSttiba1L3BD1cptNJPjSxZ8rXxF9mY3za6WA', # does not have asleep data
                'dQEFscjqnIlug8Tgq97JohhSQPG2DEOWJqS86wCrcY', 
                'HFvs2CohmhHte+AaCzFasjzegGzxZKPhkrX23iI6Xo', 
                'jgB9E8v3Z6PKdTRTCMAijBllA9YEMtrmHbe4qsbmJWw', 
                'JkY++R7E8myldLN3on6iQ78Ee78zCbrLuggfwGju3I', 
                'K4SLohf+TN1Ak8Dn8iE3Lme7rEMPISfppB2sXfHX8', 
                'oGaWetJJJEWHuvYdWYo826SQxfhCExVVQ2da8LE1Y7Q', 
                'pyt24oiDAHsmgWMvkFKz2fn2pwcHiXchd6KchLM', 
                #'PZCf1nfvhR+6fk+7+sPNMYOgb8BAMmtQtfoRS83Suc', # does not have asleep data
                'QUNCATForxzK0HHw46LrGOMWh0eVA8Y5XWEiUXX+cQ', 
                #'rIl2UK9+bQ+tzpFdbJAdbBxEa5GbgrgC030yEaENLw', 
                #'RoBW3cDOO9wWRMPO2twQff83MPc+OXn6gJ+a1DafreI', 
                'SH3kQeyd5volraxw8vOyhlowNqWBPr1IJ9URNXUL4']
                #'VVpwFNMrEglveh6MDN8lrRzTy5OwzglD4FURfM4A2is', 
                #'Wa1mcNmbh66S7VS6GIzyfCFMD3SGhbtDQyFP1ywJEsw', 
                #'XCKRE0BWRHxfP1kZIihgtT+jUjSp2GE8v5ZlhcIhVmA', 
                #'YI5Y79K6GXqAUoGP6PNyII8WKlAoel4urDxWSVVOvBw', 
                #'ypklj+8GJ15rOIH1lpKQtFJOuK+VdvyCuBPqhY3aoM', 
                #'ZSsAZ0Pq+MCqFrnjsRFn5Ua09pMCVaOV9c8ZuYb7XQY']
            
# client datasets used on the training process (25% of data)
testFolders =  [#'0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs',
                #'0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA', 
                #'2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0', 
                #'2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6TrLe89ys', 
                #['5FLZBTVAPwdq9QezHE2sVCJIs7p+r6mCemA2gp9jATk'], #does not have the file
                #'7EYF5I04EVqisUJCVNHlqn77UAuOmwL2Dahxd3cA', 
                #'a9Qgj8ENWrHvl9QqlXcIPKmyGMKgbfHk9Dbqon1HQP4', 
                #'ae4JJBZDycEcY8McJF+3BxyvZ1619y03BNdCxzpZTc', 
                #'Ch3u5Oaz96VSrQbf0z31X6jEIbeIekkC0mwPzCdeJ1U', 
                #'CH8f0yZkZL13zWuE9ks1CkVJRVrr+jsGdUXHrZ6YeA', 
                #'DHO1K4jgiwZJOfQTrxvKE2vn7hkjamigroGD5IaeRc', 
                #'DHPqzSqSttiba1L3BD1cptNJPjSxZ8rXxF9mY3za6WA', # does not have asleep data
                #'dQEFscjqnIlug8Tgq97JohhSQPG2DEOWJqS86wCrcY', 
                #'HFvs2CohmhHte+AaCzFasjzegGzxZKPhkrX23iI6Xo', 
                #'jgB9E8v3Z6PKdTRTCMAijBllA9YEMtrmHbe4qsbmJWw', 
                #'JkY++R7E8myldLN3on6iQ78Ee78zCbrLuggfwGju3I', 
                #'K4SLohf+TN1Ak8Dn8iE3Lme7rEMPISfppB2sXfHX8', 
                #'oGaWetJJJEWHuvYdWYo826SQxfhCExVVQ2da8LE1Y7Q', 
                #'pyt24oiDAHsmgWMvkFKz2fn2pwcHiXchd6KchLM', 
                #'PZCf1nfvhR+6fk+7+sPNMYOgb8BAMmtQtfoRS83Suc', # does not have asleep data
                #'QUNCATForxzK0HHw46LrGOMWh0eVA8Y5XWEiUXX+cQ', 
                'rIl2UK9+bQ+tzpFdbJAdbBxEa5GbgrgC030yEaENLw', 
                'RoBW3cDOO9wWRMPO2twQff83MPc+OXn6gJ+a1DafreI', 
                #'SH3kQeyd5volraxw8vOyhlowNqWBPr1IJ9URNXUL4'] 
                'VVpwFNMrEglveh6MDN8lrRzTy5OwzglD4FURfM4A2is', 
                'Wa1mcNmbh66S7VS6GIzyfCFMD3SGhbtDQyFP1ywJEsw', 
                'XCKRE0BWRHxfP1kZIihgtT+jUjSp2GE8v5ZlhcIhVmA', 
                'YI5Y79K6GXqAUoGP6PNyII8WKlAoel4urDxWSVVOvBw', 
                'ypklj+8GJ15rOIH1lpKQtFJOuK+VdvyCuBPqhY3aoM', 
                'ZSsAZ0Pq+MCqFrnjsRFn5Ua09pMCVaOV9c8ZuYb7XQY']

# take the list of directories and concat them
def loadDataFromFolders(foldersToLoad,inputFolders,fileType = "_transformed"):
    print(len(foldersToLoad), "datasets")
    for i in range(0,len(foldersToLoad)):
        currentFolder = foldersToLoad[i]
        print(i , "-", currentFolder,inputFolders+"student_"+currentFolder+fileType+".csv")
        #print(trainingDataSet[i])
        if(i == 0):
            temp_data = pd.read_csv(inputFolders+"student_"+currentFolder+fileType+".csv")
        else:
            dataset = pd.read_csv(inputFolders+"student_"+currentFolder+fileType+".csv")
            temp_data = pd.concat([temp_data, dataset])
    # return the dataset        
    return temp_data

In [7]:
print("Preparing test data")
 
# test data comprising 25% of the data. It must be fixed to all models being evaluated
#X_test  = pd.read_csv(inputFolders+"test/allData-classification-numeric-normalized.csv")
X_test = loadDataFromFolders(testFolders,baseFolder,"_transformed")

print()
# undestand the dataset by looking on their infos
print(X_test.info())

X_test

Preparing test data
8 datasets
0 - rIl2UK9+bQ+tzpFdbJAdbBxEa5GbgrgC030yEaENLw ../data_2019_processed/student_rIl2UK9+bQ+tzpFdbJAdbBxEa5GbgrgC030yEaENLw_transformed.csv
1 - RoBW3cDOO9wWRMPO2twQff83MPc+OXn6gJ+a1DafreI ../data_2019_processed/student_RoBW3cDOO9wWRMPO2twQff83MPc+OXn6gJ+a1DafreI_transformed.csv
2 - VVpwFNMrEglveh6MDN8lrRzTy5OwzglD4FURfM4A2is ../data_2019_processed/student_VVpwFNMrEglveh6MDN8lrRzTy5OwzglD4FURfM4A2is_transformed.csv
3 - Wa1mcNmbh66S7VS6GIzyfCFMD3SGhbtDQyFP1ywJEsw ../data_2019_processed/student_Wa1mcNmbh66S7VS6GIzyfCFMD3SGhbtDQyFP1ywJEsw_transformed.csv
4 - XCKRE0BWRHxfP1kZIihgtT+jUjSp2GE8v5ZlhcIhVmA ../data_2019_processed/student_XCKRE0BWRHxfP1kZIihgtT+jUjSp2GE8v5ZlhcIhVmA_transformed.csv
5 - YI5Y79K6GXqAUoGP6PNyII8WKlAoel4urDxWSVVOvBw ../data_2019_processed/student_YI5Y79K6GXqAUoGP6PNyII8WKlAoel4urDxWSVVOvBw_transformed.csv
6 - ypklj+8GJ15rOIH1lpKQtFJOuK+VdvyCuBPqhY3aoM ../data_2019_processed/student_ypklj+8GJ15rOIH1lpKQtFJOuK+VdvyCuBPqhY3aoM_transformed.csv


,activity,location,timestamp,time_to_next_alarm,sound,proximity,phone_lock,light,day_of_week,minutes_day,timestamp_text,class
0,0.75,1.0,0.000000e+00,0.000000,0.515992,1.0,0.0,0.000000,1.0,0.678249,2018-05-14 16:16:08+00:00,awake
1,0.25,1.0,3.211282e-07,0.000000,0.542171,0.0,1.0,0.000007,1.0,0.678944,2018-05-14 16:17:39+00:00,awake
2,0.25,1.0,6.422564e-07,0.000000,0.515992,0.0,1.0,0.000000,1.0,0.679639,2018-05-14 16:18:39+00:00,awake
3,0.00,1.0,6.422564e-07,0.000000,0.515992,0.0,1.0,0.000000,1.0,0.680334,2018-05-14 16:19:09+00:00,awake
4,0.25,1.0,6.422564e-07,0.000000,0.531341,0.0,1.0,0.000000,1.0,0.681028,2018-05-14 16:20:09+00:00,awake
...,...,...,...,...,...,...,...,...,...,...,...,...
13208,1.00,0.0,3.981050e-03,0.923909,0.577338,1.0,0.0,0.000266,1.0,0.035441,2018-06-05 00:51:21+00:00,awake
13209,1.00,0.0,3.981050e-03,0.923809,0.612109,1.0,0.0,0.000236,1.0,0.036136,2018-06-05 00:52:21+00:00,awake
13210,0.00,0.0,3.981371e-03,0.923611,0.573415,1.0,0.0,0.000118,1.0,0.037526,2018-06-05 00:54:11+00:00,awake
13211,0.75,0.0,3.981693e-03,0.923611,0.538685,1.0,0.0,0.000089,1.0,0.037526,2018-06-05 00:54:41+00:00,awake


In [8]:
print("Preparing X_train data")

# test data comprising 25% of the data. It must be fixed to all models being evaluated
#X_test  = pd.read_csv(inputFolders+"test/allData-classification-numeric-normalized.csv")
X_train = loadDataFromFolders(trainFolders,baseFolder,fileSufixTrain)

print()
# undestand the dataset by looking on their infos
print(X_train.info())

X_train

Preparing X_train data
19 datasets
0 - 0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs ../data_2019_processed/student_0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs_transformed.csv
1 - 0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA ../data_2019_processed/student_0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA_transformed.csv
2 - 2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0 ../data_2019_processed/student_2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0_transformed.csv
3 - 2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6TrLe89ys ../data_2019_processed/student_2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6TrLe89ys_transformed.csv
4 - 7EYF5I04EVqisUJCVNHlqn77UAuOmwL2Dahxd3cA ../data_2019_processed/student_7EYF5I04EVqisUJCVNHlqn77UAuOmwL2Dahxd3cA_transformed.csv
5 - a9Qgj8ENWrHvl9QqlXcIPKmyGMKgbfHk9Dbqon1HQP4 ../data_2019_processed/student_a9Qgj8ENWrHvl9QqlXcIPKmyGMKgbfHk9Dbqon1HQP4_transformed.csv
6 - ae4JJBZDycEcY8McJF+3BxyvZ1619y03BNdCxzpZTc ../data_2019_processed/student_ae4JJBZDycEcY8McJF+3BxyvZ1619y03BNdCxzpZTc_transformed.csv
7 - Ch3u

,activity,location,timestamp,time_to_next_alarm,sound,proximity,phone_lock,light,day_of_week,minutes_day,timestamp_text,class
0,0.00,0.0,0.000000e+00,0.982143,0.000000,1.0,0.0,0.000617,0.0,0.597637,2018-05-15 14:20:45+00:00,awake
1,0.00,0.0,0.000000e+00,0.982143,0.000000,1.0,0.0,0.000617,0.0,0.597637,2018-05-15 14:20:45+00:00,awake
2,0.25,0.5,3.211923e-07,0.982044,0.604408,1.0,0.0,0.000583,0.0,0.598332,2018-05-15 14:21:15+00:00,awake
3,0.25,0.5,3.211923e-07,0.982044,0.604408,1.0,0.0,0.005117,0.0,0.598332,2018-05-15 14:21:45+00:00,awake
4,0.25,0.5,3.211923e-07,0.981944,0.601849,1.0,0.0,0.000700,0.0,0.599027,2018-05-15 14:22:15+00:00,awake
...,...,...,...,...,...,...,...,...,...,...,...,...
10412,0.25,1.0,5.354495e-03,0.013095,0.569798,1.0,0.0,0.000000,1.0,0.408617,2018-06-11 09:48:21+00:00,awake
10413,0.25,1.0,5.354495e-03,0.013194,0.569798,1.0,0.0,0.000000,1.0,0.408617,2018-06-11 09:48:51+00:00,awake
10414,0.25,1.0,5.354495e-03,0.030853,0.517226,1.0,1.0,0.000000,1.0,0.409312,2018-06-11 09:49:21+00:00,awake
10415,0.25,1.0,5.355459e-03,0.030159,0.517226,1.0,1.0,0.000000,1.0,0.414177,2018-06-11 09:56:17+00:00,awake


In [9]:
#X_train = pd.read_csv(baseFolder+"train/allData-classification-numeric-normalized.csv")
#X_test  = pd.read_csv(baseFolder+"test/allData-classification-numeric-normalized.csv")
#X_train = pd.read_csv(baseFolder+"train/allData-classification-numeric-normalized_balanced_undersample.csv")
#X_test  = pd.read_csv(baseFolder+"test/allData-classification-numeric-normalized_balanced_oversample.csv")

#AA = pd.read_csv(baseFolder+"allData-classification-numeric-normalized.csv")
#X_train, X_test = train_test_split(AA,test_size=0.25)

In [10]:
print(X_train.info())
X_train

<class 'pandas.core.frame.DataFrame'>
Index: 347067 entries, 0 to 10416
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   activity            347067 non-null  float64
 1   location            347067 non-null  float64
 2   timestamp           347067 non-null  float64
 3   time_to_next_alarm  347067 non-null  float64
 4   sound               347067 non-null  float64
 5   proximity           347067 non-null  float64
 6   phone_lock          347067 non-null  float64
 7   light               347067 non-null  float64
 8   day_of_week         347067 non-null  float64
 9   minutes_day         347067 non-null  float64
 10  timestamp_text      347067 non-null  object 
 11  class               347067 non-null  object 
dtypes: float64(10), object(2)
memory usage: 34.4+ MB
None


,activity,location,timestamp,time_to_next_alarm,sound,proximity,phone_lock,light,day_of_week,minutes_day,timestamp_text,class
0,0.00,0.0,0.000000e+00,0.982143,0.000000,1.0,0.0,0.000617,0.0,0.597637,2018-05-15 14:20:45+00:00,awake
1,0.00,0.0,0.000000e+00,0.982143,0.000000,1.0,0.0,0.000617,0.0,0.597637,2018-05-15 14:20:45+00:00,awake
2,0.25,0.5,3.211923e-07,0.982044,0.604408,1.0,0.0,0.000583,0.0,0.598332,2018-05-15 14:21:15+00:00,awake
3,0.25,0.5,3.211923e-07,0.982044,0.604408,1.0,0.0,0.005117,0.0,0.598332,2018-05-15 14:21:45+00:00,awake
4,0.25,0.5,3.211923e-07,0.981944,0.601849,1.0,0.0,0.000700,0.0,0.599027,2018-05-15 14:22:15+00:00,awake
...,...,...,...,...,...,...,...,...,...,...,...,...
10412,0.25,1.0,5.354495e-03,0.013095,0.569798,1.0,0.0,0.000000,1.0,0.408617,2018-06-11 09:48:21+00:00,awake
10413,0.25,1.0,5.354495e-03,0.013194,0.569798,1.0,0.0,0.000000,1.0,0.408617,2018-06-11 09:48:51+00:00,awake
10414,0.25,1.0,5.354495e-03,0.030853,0.517226,1.0,1.0,0.000000,1.0,0.409312,2018-06-11 09:49:21+00:00,awake
10415,0.25,1.0,5.355459e-03,0.030159,0.517226,1.0,1.0,0.000000,1.0,0.414177,2018-06-11 09:56:17+00:00,awake


In [11]:
# transform output to one_hot_encoding for the testing dataset
X_test = transform_output_nominal_class_into_one_hot_encoding(X_test)

# transform output to one_hot_encoding for the testing dataset
X_train = transform_output_nominal_class_into_one_hot_encoding(X_train)


# transforms the input data to float32
X_test = transform_data_type(X_test)

# transforms the input data to float32
X_train = transform_data_type(X_train)

In [12]:
X_train

,activity,location,timestamp,time_to_next_alarm,sound,proximity,phone_lock,light,day_of_week,minutes_day,timestamp_text,class,awake,asleep
0,0.00,0.0,0.000000e+00,0.982143,0.000000,1.0,0.0,0.000617,0.0,0.597637,2018-05-15 14:20:45+00:00,awake,1.0,0.0
1,0.00,0.0,0.000000e+00,0.982143,0.000000,1.0,0.0,0.000617,0.0,0.597637,2018-05-15 14:20:45+00:00,awake,1.0,0.0
2,0.25,0.5,3.211923e-07,0.982044,0.604408,1.0,0.0,0.000583,0.0,0.598332,2018-05-15 14:21:15+00:00,awake,1.0,0.0
3,0.25,0.5,3.211923e-07,0.982044,0.604408,1.0,0.0,0.005117,0.0,0.598332,2018-05-15 14:21:45+00:00,awake,1.0,0.0
4,0.25,0.5,3.211923e-07,0.981944,0.601849,1.0,0.0,0.000700,0.0,0.599027,2018-05-15 14:22:15+00:00,awake,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10412,0.25,1.0,5.354495e-03,0.013095,0.569798,1.0,0.0,0.000000,1.0,0.408617,2018-06-11 09:48:21+00:00,awake,1.0,0.0
10413,0.25,1.0,5.354495e-03,0.013194,0.569798,1.0,0.0,0.000000,1.0,0.408617,2018-06-11 09:48:51+00:00,awake,1.0,0.0
10414,0.25,1.0,5.354495e-03,0.030853,0.517226,1.0,1.0,0.000000,1.0,0.409312,2018-06-11 09:49:21+00:00,awake,1.0,0.0
10415,0.25,1.0,5.355459e-03,0.030159,0.517226,1.0,1.0,0.000000,1.0,0.414177,2018-06-11 09:56:17+00:00,awake,1.0,0.0


In [13]:
# selects the data to train and test
X_train_data_s = pd.DataFrame(data=X_train,columns=inputFeatures)
y_train_data_s = pd.DataFrame(data=X_train,columns=outputClasses)
# selec test dataset (fixed to all)
X_test_data_s = pd.DataFrame(data=X_test,columns=inputFeatures)
y_test_data_s = pd.DataFrame(data=X_test,columns=outputClasses)

In [14]:
X_test_data_s

,activity,location,day_of_week,light,phone_lock,proximity,sound,time_to_next_alarm,minutes_day
0,0.75,1.0,1.0,0.000000,0.0,1.0,0.515992,0.000000,0.678249
1,0.25,1.0,1.0,0.000007,1.0,0.0,0.542171,0.000000,0.678944
2,0.25,1.0,1.0,0.000000,1.0,0.0,0.515992,0.000000,0.679639
3,0.00,1.0,1.0,0.000000,1.0,0.0,0.515992,0.000000,0.680334
4,0.25,1.0,1.0,0.000000,1.0,0.0,0.531341,0.000000,0.681028
...,...,...,...,...,...,...,...,...,...
13208,1.00,0.0,1.0,0.000266,0.0,1.0,0.577338,0.923909,0.035441
13209,1.00,0.0,1.0,0.000236,0.0,1.0,0.612109,0.923810,0.036136
13210,0.00,0.0,1.0,0.000118,0.0,1.0,0.573415,0.923611,0.037526
13211,0.75,0.0,1.0,0.000089,0.0,1.0,0.538685,0.923611,0.037526


In [15]:
y_test_data_s.shape

(110603, 2)

In [16]:
print("Transform the data to series")

X_train_data, y_train_data = create_dataset_time_series_with_one_output_foward(   #timestamp
    X_train_data_s, 
    y_train_data_s, 
    TIME_SERIES_SIZE, 
    TIME_STEP_SHIFT
)

X_test_data, y_test_data = create_dataset_time_series_with_one_output_foward(    #timestamp
    X_test_data_s, 
    y_test_data_s, 
    TIME_SERIES_SIZE, 
    TIME_STEP_SHIFT
)

print("shape: ",X_train_data.shape, y_train_data.shape)
print("Size: ",X_test_data.shape,y_test_data.shape)       

Transform the data to series
shape:  (11567, 60, 9) (11567, 2)
Size:  (3685, 60, 9) (3685, 2)


In [17]:
# transtorm data to tensor slices
test_dataset_series = tf.data.Dataset.from_tensor_slices((X_test_data, y_test_data))
train_dataset_series = tf.data.Dataset.from_tensor_slices((X_train_data, y_train_data))

2024-02-23 22:20:43.967050: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-23 22:20:44.043995: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2024-02-23 22:20:44.044019: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-02-23 22:20:44.046484: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (o

In [18]:
train_dataset_series

<TensorSliceDataset element_spec=(TensorSpec(shape=(60, 9), dtype=tf.float32, name=None), TensorSpec(shape=(2,), dtype=tf.float32, name=None))>

In [19]:
# batch_size
train_dataset_series = train_dataset_series.batch(BATCH_SIZE)

train_dataset_series

<BatchDataset element_spec=(TensorSpec(shape=(None, 60, 9), dtype=tf.float32, name=None), TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))>

In [20]:
# batch_size
validation_dataset_series = test_dataset_series.batch(BATCH_SIZE)

validation_dataset_series

<BatchDataset element_spec=(TensorSpec(shape=(None, 60, 9), dtype=tf.float32, name=None), TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))>

In [21]:
MAX_EPOCH = max(EPOCHS_ARRAY_TEST)

MAX_EPOCH

4

In [22]:
print("configure checkpoint")
filepath = outputCheckpointFolder + "/" + checkpointName_prefix
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=VERBOSE, save_best_only=False, mode='auto')
#WARNING:tensorflow:`period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.


configure checkpoint


In [23]:
#generate model
model = keras.Sequential()
model.add(Bidirectional(LSTM(128,return_sequences=True),input_shape=(TIME_SERIES_SIZE, 9,)))
model.add((LSTM(128)))
model.add(keras.layers.Dense(2, activation='softmax'))#softmax,sigmoid

# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalAccuracy()])
          #loss='binary_crossentropy',loss='categorical_crossentropy',
          #loss='binary_crossentropy',  sparse_categorical_crossentropy 
        
print("input_shape=[", X_train_data.shape,"]")
print("output shape:",len(outputClasses))
print("Epochs:", MAX_EPOCH)

current_time = datetime.datetime.now()
time_stamp = current_time.timestamp()
print("Start timestamp:", time_stamp,current_time)
print()

verbose, epochs, batch_size = VERBOSE, MAX_EPOCH, BATCH_SIZE  
# fit network
history = model.fit(train_dataset_series, epochs=epochs, verbose=verbose, 
                    batch_size=batch_size, callbacks=[checkpoint],
                    validation_data=validation_dataset_series) #, batch_size=batch_size, validation_split=0.1

# generate time metrics
current_time2 = datetime.datetime.now()
time_stamp2 = current_time2.timestamp()
processing_time_s = (time_stamp2-time_stamp)
print("End timestamp:", time_stamp2,current_time2)
print("Processing time (s):", (processing_time_s))
print("Processing time (m):", (processing_time_s/60))
print("Processing time (h):", ((processing_time_s/60)/60))

input_shape=[ (11567, 60, 9) ]
output shape: 2
Epochs: 4
Start timestamp: 1708726847.940412 2024-02-23 22:20:47.940412

End timestamp: 1708726985.551331 2024-02-23 22:23:05.551331
Processing time (s): 137.61091899871826
Processing time (m): 2.2935153166453044
Processing time (h): 0.03822525527742174


In [24]:
dfhistory = pd.DataFrame(data=history.history) 

print("print all loss, ",len(dfhistory))
history

outputHistoryFilepath = outputCheckpointFolder+"/train_metrics_history.csv"
print("save all loss on ",outputHistoryFilepath)
dfhistory.to_csv(outputHistoryFilepath, sep=',', encoding='utf-8', index=False)

# outputCheckpointFolder
#history.history.to_csv(outputMetricFile, sep=',', encoding='utf-8', index=False)
# loss and val_loss
# acc and val_acc
history.history["loss"]
history.history["categorical_accuracy"]

dfhistory

print all loss,  4
save all loss on  result_trad_LSTM_unb_batch_size_32_window_60-30_checkpoints/train_metrics_history.csv


,loss,categorical_accuracy,val_loss,val_categorical_accuracy
0,0.490279,0.783436,0.484446,0.784532
1,0.470544,0.799084,0.473977,0.775305
2,0.464294,0.796317,0.474620,0.774220
3,0.457442,0.798219,0.481579,0.774220


In [25]:
columnsOutputMetrics = ['NN_type','units','epochs','batch_size','window_size','time_step_shift',
           'start_time','end_time','time_s','time_m', "train_accuracy", "train_loss", "val_accuracy", "val_loss",
           'class','accuracy','precision','recall','f1_score','cohen_kappa_score','roc_auc_score','confusion_matrix',
           'TP','FP','FN','TN']

allMetrics = []

In [27]:
#generate model
model = keras.Sequential()
model.add(Bidirectional(LSTM(128,return_sequences=True),
      input_shape=[X_train_data.shape[1], X_train_data.shape[2]]))
model.add((LSTM(128)))
model.add(keras.layers.Dense(2, activation='softmax'))#softmax,sigmoid

# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalAccuracy()])
          #loss='binary_crossentropy',loss='categorical_crossentropy',
          #loss='binary_crossentropy',  sparse_categorical_crossentropy   

for TEST_EPOCHS in EPOCHS_ARRAY_TEST:
    # general data from the run
    generalData = [NN_type,UNITS_NUMBER,TEST_EPOCHS,BATCH_SIZE,TIME_SERIES_SIZE,TIME_STEP_SHIFT]
    
    print("input_shape=[", X_train_data.shape,"]")
    print("output shape:",len(outputClasses))
    print("Epochs:",TEST_EPOCHS)
    
    current_time = datetime.datetime.now()
    time_stamp = current_time.timestamp()
    print("Start timestamp:", time_stamp,current_time)
    print()
    
    checkPpath = filepath.replace("{epoch}",str(TEST_EPOCHS))
    print("Loading checkpoint: ", checkPpath)
    print('')
    
    model.load_weights(checkPpath)
    # evaluate model
    #accuracy = model.evaluate(test_dataset_series1) # , batch_size=batch_size, verbose=0
    # predict
    yhat_probs = model.predict(X_test_data,verbose=VERBOSE)
    # predict crisp classes for test set deprecated
    
    # generate time metrics
    current_time2 = datetime.datetime.now()
    time_stamp2 = current_time2.timestamp()
    processing_time_s = (time_stamp2-time_stamp)
    
    train_accuracy = history.history["loss"][TEST_EPOCHS-1]
    train_loss     = history.history["categorical_accuracy"][TEST_EPOCHS-1]
    val_acc = history.history["val_loss"][TEST_EPOCHS-1]
    val_loss     = history.history["val_categorical_accuracy"][TEST_EPOCHS-1]
    # generate general metrics
    rowData = [current_time,current_time2,processing_time_s,(processing_time_s)/60,train_accuracy,train_loss,val_acc,val_loss]

    y_pred_labels = pd.DataFrame(data=yhat_probs,columns=['awake','asleep'])
    y_test_labels = pd.DataFrame(data=y_test_data,columns=['awake','asleep'])

    feature_metrics_gathered = []
    # print('')
    print('awake')    
    res,resA = printMetrics(y_test_labels['awake'],y_pred_labels['awake'])
    feature_metrics_gathered.append(res)
   
    #columns = ['NN_type','units','epochs','batch_size','max_iterations',''Users',
    #            round_iteration','start_time','end_time','round_time_s','round_time_m',
    #           'class','accuracy','precision','recall','f1_score','cohen_kappa_score','roc_auc_score','confusion_matrix',
    #           'TP','FP','FN','TN']
    # new data
    classData = np.concatenate((['awake'], resA))
    classData = np.concatenate((rowData, classData))
    classData = np.concatenate((generalData, classData))
    allMetrics.append(classData)
    
    print('')
    print('asleep')
    res,resA = printMetrics(y_test_labels['asleep'],y_pred_labels['asleep'])
    feature_metrics_gathered.append(res)
    # new data
    classData = np.concatenate((['asleep'], resA))
    classData = np.concatenate((rowData, classData))
    classData = np.concatenate((generalData, classData))
    allMetrics.append(classData)
    print('')
    print('Global')
    resA = showGlobalMetrics(feature_metrics_gathered) #return [accuracy,precision,recall,f1_score,cohen_kappa_score,roc_auc_score
    # new data
    classData = np.concatenate((['avg'], resA))
    classData = np.concatenate((rowData, classData))
    classData = np.concatenate((generalData, classData))
    allMetrics.append(classData)
    print('')
    print("End timestamp:", time_stamp2,current_time2)
    print('')
    print('')
    print('-----------------------------------------------------------------------')
    print('')
    print('')

input_shape=[ (11567, 60, 9) ]
output shape: 2
Epochs: 1
Start timestamp: 1708727121.608008 2024-02-23 22:25:21.608008

Loading checkpoint:  result_trad_LSTM_unb_batch_size_32_window_60-30_checkpoints/checkpoint_epoch_1.hdf5

awake
Accuracy: 0.784532
Precision: 0.784943
Recall: 0.994041
F1 score: 0.877204
Cohens kappa: 0.088914
ROC AUC: 0.724247
\Confusion Matrix
[[  55  777]
 [  17 2836]]

asleep
Accuracy: 0.784532
Precision: 0.763889
Recall: 0.066106
F1 score: 0.121681
Cohens kappa: 0.088914
ROC AUC: 0.724247
\Confusion Matrix
[[2836   17]
 [ 777   55]]

Global
2
accuracy:  0.7845318860244234
precision:  0.7744160746686348
recall:  0.5300735646013642
f1_score:  0.4994426256880784
cohen_kappa_score:  0.08891422891719325
roc_auc_score:  0.724247123473267

End timestamp: 1708727125.567278 2024-02-23 22:25:25.567278


-----------------------------------------------------------------------


input_shape=[ (11567, 60, 9) ]
output shape: 2
Epochs: 4
Start timestamp: 1708727125.59734 2024-02

In [28]:
dataMetrics = pd.DataFrame(data=allMetrics,columns=columnsOutputMetrics) 

dataMetrics

,NN_type,units,epochs,batch_size,window_size,time_step_shift,start_time,end_time,time_s,time_m,...,precision,recall,f1_score,cohen_kappa_score,roc_auc_score,confusion_matrix,TP,FP,FN,TN
0,LSTM,128,1,32,60,30,2024-02-23 22:25:21.608008,2024-02-23 22:25:25.567278,3.959270,0.065988,...,0.7849432604483808,0.9940413599719593,0.8772038354469532,0.08891422891719325,0.724247123473267,0.0,55.0,777.0,17.0,2836.0
1,LSTM,128,1,32,60,30,2024-02-23 22:25:21.608008,2024-02-23 22:25:25.567278,3.959270,0.065988,...,0.7638888888888888,0.06610576923076923,0.12168141592920355,0.08891422891719325,0.724247123473267,0.0,2836.0,17.0,777.0,55.0
2,LSTM,128,1,32,60,30,2024-02-23 22:25:21.608008,2024-02-23 22:25:25.567278,3.959270,0.065988,...,0.7744160746686348,0.5300735646013642,0.4994426256880784,0.08891422891719325,0.724247123473267,None,None,None,None,None
3,LSTM,128,4,32,60,30,2024-02-23 22:25:25.597340,2024-02-23 22:25:28.711278,3.113938,0.051899,...,0.7742198100407056,1.0,0.8727439583970633,0.0,0.7312591839898621,0.0,0.0,832.0,0.0,2853.0
4,LSTM,128,4,32,60,30,2024-02-23 22:25:25.597340,2024-02-23 22:25:28.711278,3.113938,0.051899,...,0.0,0.0,0.0,0.0,0.7312591839898621,0.0,2853.0,0.0,832.0,0.0
5,LSTM,128,4,32,60,30,2024-02-23 22:25:25.597340,2024-02-23 22:25:28.711278,3.113938,0.051899,...,0.3871099050203528,0.5,0.43637197919853166,0.0,0.7312591839898621,None,None,None,None,None


In [ ]:
dataMetrics.to_csv(outputMetricFile, sep=',', encoding='utf-8', index=False)